In [1]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


You should consider upgrading via the 'C:\Users\Deniz\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install stable-baselines3[extra] protobuf==3.20.*

You should consider upgrading via the 'C:\Users\Deniz\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install mss pydirectinput pytesseract gym

You should consider upgrading via the 'C:\Users\Deniz\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import numpy as np
from mss import mss
import pydirectinput
import pytesseract
from matplotlib import pyplot as plt
import time
from gym import Env
from gym.spaces import Box, Discrete, MultiDiscrete, Tuple
import copy
import random
import pygame
import threading
import os 
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker
import sys
from IPython import display
from stable_baselines3 import DQN, A2C, PPO
import matplotlib.pyplot as plt

In [5]:
def list_to_array(list):
    updated_list = []
    for sublist in list:
        updated_sublist = []
        for i in sublist:
            updated_sublist.append(i+1)
        updated_list.append(updated_sublist)
    for sublist in updated_list:
        while len(sublist)<4:
            sublist.append(0)
    while len(updated_list) < 8:
        updated_list.append([-1,-1,-1,-1])
    return np.array(updated_list)

In [6]:
class WaterSortAgent(Env):
    def __init__(self):
        self.level = 0
        self.tubes_number = 4
        self.new_game = True
        self.tube_colors = []
        self.initial_colors = []
        self.seed = 0
        self.selected = False
        self.tube_rects = []
        self.select_rect = 100
        self.win = False

        self.action_number = 0
        self.invalid_action_counter = 0
        self.last_three_observation = []

        #self.plot_scores = []
        #self.mean_score = []
        #self.total_score = 0
        #self.last_100_score = 0
        #plt.ion()
        
        self.observation_space = Box(low=-1, high=6, shape=(8, 4), dtype=int)
        self.action_space = MultiDiscrete([8, 8])
    
    def generate_start(self):
        random.seed(self.seed+self.level+12)
        if self.level==0:
            self.tubes_number = 4
        elif self.level==1:
            self.tubes_number = 5
        elif self.level==2:
            self.tubes_number = 6
        elif self.level==3:
            self.tubes_number = 7
        else:
            self.tubes_number = 8
        tubes_colors = []
        available_colors = []
        for i in range(self.tubes_number):
            tubes_colors.append([])
            if i < self.tubes_number - 2:
                for j in range(4):
                    available_colors.append(i)
        for i in range(self.tubes_number - 2):
            for j in range(4):
                color = random.choice(available_colors)
                tubes_colors[i].append(color)
                available_colors.remove(color)
        self.level +=1
        self.seed +=1

        return tubes_colors

    def calc_move(self, colors, selected_rect, destination):
        if len(colors) > selected_rect and len(colors) > destination:
            chain = True
            color_on_top = 100
            length = 1
            color_to_move = 100
            if len(colors[selected_rect]) > 0:
                color_to_move = colors[selected_rect][-1]
                for i in range(1, len(colors[selected_rect])):
                    if chain:
                        if colors[selected_rect][-1 - i] == color_to_move:
                            length += 1
                        else:
                            chain = False
            if 4 > len(colors[destination]):
                if len(colors[destination]) == 0:
                    color_on_top = color_to_move
                else:
                    color_on_top = colors[destination][-1]
            if color_on_top == color_to_move:
                for i in range(length):
                    if len(colors[destination]) < 4:
                        if len(colors[selected_rect]) > 0:
                            colors[destination].append(color_on_top)
                            colors[selected_rect].pop(-1)

        return colors

    def check_victory(self, colors):
        won = True
        for i in range(len(colors)):
            if len(colors[i]) > 0:
                if len(colors[i]) != 4:
                    won = False
                else:
                    main_color = colors[i][-1]
                    for j in range(len(colors[i])):
                        if colors[i][j] != main_color:
                            won = False
        return won

    def get_done(self):
        done = False
        if self.invalid_action_counter == 1000:
            done = True
        return done

    def step(self, action):
        self.action_number += 1
        action_type = None
        reward = 0
        done = False
        info = {}

        if len(self.last_three_observation) == 0:
            self.last_three_observation.append(self.tube_colors)
        obs_list = copy.deepcopy(self.last_three_observation)

        self.tube_colors = self.calc_move(self.tube_colors, action[0], action[1])
        observation = copy.deepcopy(self.tube_colors)

        if self.check_victory(observation):
            reward = 10
            self.reward += reward
            done = True
            action_type = "WON"

        else:
            if len(obs_list) == 4:
                obs_list.pop(0)

            if sorted(obs_list[-1]) == sorted(observation):
                reward = 0
                action_type = "Not Changed"
                self.invalid_action_counter += 1

            elif len(obs_list) == 2 and sorted(obs_list[-2]) == sorted(observation):
                reward = 0
                action_type = "Iteration"
                self.invalid_action_counter += 1

            elif len(obs_list) == 3 and (sorted(obs_list[-2]) == sorted(observation) or sorted(obs_list[-3]) == sorted(observation)):
                reward = 0
                action_type = "Iteration"
                self.invalid_action_counter += 1

            elif self.check_fullcolor(observation) > self.check_fullcolor(obs_list[-1]):
                reward = 5
                action_type = "Fullcolor"
                obs_list.append(observation)

            else:
                reward = 1
                action_type = "Valid Action"
                obs_list.append(observation)

            self.reward += reward
        if done == False:
            done = self.get_done()
        self.last_three_observation = copy.deepcopy(obs_list)

        #print(action, action_type, "ActionCounter:", self.action_number, "Reward:", reward)
        #print(observation)
        #print()
        return list_to_array(observation), reward, done, info
    

    def reset(self):
        """if self.level != 0:
            self.last_100_score += self.reward
            if self.level % 100 == 99:
                self.plot_scores.append(self.last_100_score/100)
                self.total_score += self.last_100_score
                self.last_100_score = 0
                self.mean_score.append(self.total_score/(self.level))"""
        self.reward = 0
        observation = self.generate_start()
        while self.check_victory(observation) == True:
            observation = self.generate_start()
            self.level -= 1
        self.action_number = 0
        self.invalid_action_counter = 0
        self.last_three_observation = []
        self.tube_colors = observation

        #print(observation)
        #print()
        return list_to_array(observation)

    def check_fullcolor(self, obs):
        x = 0
        for i in obs:
            if len(i) == 4:
                if all(x == i[0] for x in i):
                    x += 1
        return x
                
    

In [56]:
import time
env = WaterSortAgent()
for episode in range(5):
    obs = env.reset()
    done = False  
    total_reward   = 0
    while not done: 
        obs, reward,  done, info =  env.step(env.action_space.sample())
        total_reward  += reward
    print('Total Reward: {}, Level: {}, Action: {}, InvalidAction: {},'.format(total_reward, env.level, 
                                                                       env.action_number, env.invalid_action_counter))   

Total Reward: 22, Level: 1, Action: 257, InvalidAction: 248,
Total Reward: 31, Level: 2, Action: 293, InvalidAction: 279,
Total Reward: 36, Level: 3, Action: 428, InvalidAction: 413,
Total Reward: 46, Level: 4, Action: 282, InvalidAction: 261,
Total Reward: 58, Level: 5, Action: 593, InvalidAction: 564,


In [57]:
model1= A2C.load('models/model_level_1.zip') 
model2= A2C.load('models/model_level_2.zip') 
model3= A2C.load('models/model_level_3.zip') 
model4= A2C.load('models/model_level_4.zip') 
model5= A2C.load('models/model_level_5.zip')

In [74]:
env = WaterSortAgent()
for episode in range(5):
    obs = env.reset()
    if env.tubes_number == 4:
        model = model1
    elif env.tubes_number == 5:
        model = model2
    elif env.tubes_number == 6:
        model = model3
    elif env.tubes_number == 7:
        model = model4
    else:
        model = model5
    done = False  
    total_reward   = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        total_reward  += reward
    print('Total Reward: {}, Level: {}, Action: {}, InvalidAction: {},'.format(total_reward, env.level, 
                                                                       env.action_number, env.invalid_action_counter))   

Total Reward: 19, Level: 1, Action: 6, InvalidAction: 0,
Total Reward: 28, Level: 2, Action: 12, InvalidAction: 1,
Total Reward: 36, Level: 3, Action: 143, InvalidAction: 128,
Total Reward: 44, Level: 4, Action: 146, InvalidAction: 127,
Total Reward: 50, Level: 5, Action: 496, InvalidAction: 475,
